In [23]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

data = pd.read_csv("bitcoin_data_last_year.csv", parse_dates=["timestamp"])
data.set_index("timestamp", inplace=True)

closing_prices = data["close"].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(closing_prices)

def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(seq_length, len(data)):
        sequences.append(data[i-seq_length:i, 0])
        labels.append(data[i, 0])
    return np.array(sequences), np.array(labels)

seq_length = 60
X, y = create_sequences(scaled_data, seq_length)

X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [24]:
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

early_stop = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)

train_size = int(len(X) * 0.8)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]


/opt/homebrew/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [26]:
history = model.fit(
    X_train, y_train,
    epochs=1,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop, reduce_lr]
)


13139/13139 ━━━━━━━━━━━━━━━━━━━━ 769s 59ms/step - loss: 8.5953e-04 - val_loss: 2.0581e-05 - learning_rate: 0.0010


In [34]:
predicted_prices = model.predict(X_val)

predicted_prices = scaler.inverse_transform(predicted_prices)
y_val = scaler.inverse_transform([y_val])

plt.figure(figsize=(14, 5))
plt.plot(y_val[0], color='blue', label="Prix Réels")
plt.plot(predicted_prices, color='red', label="Prédictions LSTM")
plt.title("Prédiction des prix du Bitcoin avec LSTM")
plt.xlabel("Temps")
plt.ylabel("Prix du Bitcoin")
plt.legend()
plt.show()


3285/3285 ━━━━━━━━━━━━━━━━━━━━ 49s 15ms/step


ValueError: Found array with dim 3. None expected <= 2.

In [29]:
import pickle

# Sauvegarde du modèle en format Keras h5
model.save("bitcoin_lstm_model.h5")

# Sauvegarde du scaler pour utiliser les mêmes transformations sur les nouvelles données
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (fsevents)
0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/homebrew/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/op

SystemExit: 1

/opt/homebrew/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
